<a href="https://colab.research.google.com/github/alexagalicia/codepath-prework/blob/main/Data_Merging.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime

Format DateTime objects

In [ ]:
string_date_format = "%Y-%m-%d"

Load Datasets

In [ ]:
fsd = pd.read_csv('franchisee_service_data.csv')
ad = pd.read_csv('cleaned_admin_data.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'franchisee_service_data.csv'

Standardize Naming Convetion of clean franchisee dataset

In [ ]:
#this will standardize everything and fixes other typos

group_mapping = {
    # Standardize YESCO capitalization
    'Yesco Ontario Southwest': 'YESCO Ontario Southwest',
    'Yesco Calgary': 'YESCO Calgary',
    'Yesco Chicago': 'YESCO Chicago',
    'Yesco Cleveland': 'YESCO Cleveland',
    'Yesco Colorado Springs': 'YESCO Colorado Springs',
    'Yesco Concord - Nashua': 'YESCO Concord - Nashua',
    'Yesco Denver': 'YESCO Denver',
    'Yesco Fort Lauderdale': 'YESCO Fort Lauderdale',
    'Yesco Fort Myers': 'YESCO Fort Myers',
    'Yesco Hampton Roads': 'YESCO Hampton Roads',
    'Yesco Louisville': 'YESCO Louisville',
    'Yesco Mid Atlantic': 'YESCO Mid Atlantic',
    'Yesco Mid-Atlantic': 'YESCO Mid Atlantic',  # Consolidate the hyphen variant
    'Yesco Nashville / Florida Panhandle': 'YESCO Nashville / Florida Panhandle',
    'Yesco New Jersey Central & Coast': 'YESCO New Jersey Central & Coast',
    'Yesco New Jersey Central & South': 'YESCO New Jersey Central & South',
    'Yesco North Carolina Central': 'YESCO North Carolina Central',
    'Yesco North Carolina East': 'YESCO North Carolina East',
    'Yesco Oklahoma / New England South': 'YESCO Oklahoma / New England South',
    'Yesco Orlando North': 'YESCO Orlando North',
    'Yesco Wilchita': 'YESCO Wichita',  # Fix typo
    'Yesco Wisconsin Central': 'YESCO Wisconsin Central',
    'Yesco Whitby': 'YESCO Whitby',

    # Keep these as-is (already correct format)
    'Franchising Team': 'Franchising Team',
    'Px Improvement': 'PX Improvement',
    'Wiin': 'WIIN',
    'Yesco8 Cincinnati - Dayton': 'YESCO Cincinnati - Dayton',  # Remove the "8"
    'Hampton Roads': 'YESCO Hampton Roads',  # Add YESCO prefix
    'Yesco Hampton Roads / Richmond': 'YESCO Hampton Roads / Richmond',
    'Yesco Hampton Roads South / Dallas South': 'YESCO Hampton Roads South / Dallas South',
    'Yesco Myrtle Beach': 'YESCO Myrtle Beach',
}

# Apply the mapping
fsd['Tenant Name'] = fsd['Tenant Name'].replace(group_mapping)

print("Standardized Group names")
print("\nUnique groups after cleaning:")
# Filter out NaN values before sorting
unique_groups_cleaned = [group for group in fsd['Tenant Name'].unique() if isinstance(group, str)]
print(sorted(unique_groups_cleaned))
print(f"\nTotal unique groups: {len(unique_groups_cleaned)}")

Further Clean Admin Data to be in the format:

```
Orginization | Date | Stage #
```
Where Stage # is a numerical representation of how far through the program they have made it



In [ ]:
# Select the relevant columns for stage calculation
stage_columns = ad.columns[ad.columns.get_loc('Setup Vision'):]

# Function to determine the stage number
def get_stage(row):
    stage = 0
    for i, col in enumerate(stage_columns):
        if row[col] == 2:
            stage = i + 1  # Stages are 1-indexed based on the column order
    return stage

# Apply the function to create the 'Stage #' column
ad['Stage'] = ad.apply(get_stage, axis=1)

# Select only the required columns
cleaned_ad = ad[['Group', 'Date', 'Stage']]

# Filter out rows where Organization name does not start with "YESCO"
cleaned_ad = cleaned_ad[cleaned_ad['Group'].str.startswith('YESCO', na=False)].copy()
cleaned_ad = cleaned_ad.rename(columns={"Group": "Organization"})

# Display the cleaned DataFrame
display(cleaned_ad.head())

In [ ]:
# Sort the cleaned_ad DataFrame by 'Organization' and 'Date'
sorted_cleaned_ad = cleaned_ad.sort_values(by=['Organization', 'Date'])

# Calculate the cumulative maximum stage for each organization
sorted_cleaned_ad['Stage'] = sorted_cleaned_ad.groupby('Organization')['Stage'].cummax()

# Display the sorted DataFrame with cumulative maximum stage
ad = sorted_cleaned_ad
display(ad.head())

Create a new data frame to be in the following format to encapsulate all important information for analysis


```
ID | Name | Stage @ Point in Time | Date | Subtotal
```



In [ ]:
fsd['Date Created'] = pd.to_datetime(fsd['Date Created'])
ad['Date'] = pd.to_datetime(ad['Date'])

fsd = fsd.rename(columns={"Tenant ID": "ID", "Tenant Name": "Organization", "Date Created": "Date", "Subtotal": "Sale"})
fsd = fsd.drop('Servizio ID', axis=1)

display(fsd)

In [ ]:
# Merge fsd and ad on 'Organization'
# We use a left merge on fsd to keep all sales data
# We merge based on the date such that we get the stage at or before the sale date
df = pd.merge_asof(fsd.sort_values('Date'), ad.sort_values('Date'), on='Date', by='Organization', direction='backward')

# Display the resulting dataframe
display(df)

Visual Analysis of the Data

In [ ]:
org_name = 'YESCO Louisville'
org_df = df[df['Organization'] == org_name].copy()

# Find the date when stage 1 was reached
stage_1_date = ad[ad['Organization'] == org_name][ad['Stage'] >= 1]['Date'].min()

# Filter data before reaching stage 1
org_df_before_stage_1 = org_df[org_df['Date'] < stage_1_date].copy()

# Filter data after reaching stage 1
org_df_after_stage_1 = org_df[org_df['Date'] >= stage_1_date].copy()

# Calculate weekly revenue before stage 1
org_df_before_stage_1['Week'] = org_df_before_stage_1['Date'].dt.to_period('W')
weekly_revenue_before = org_df_before_stage_1.groupby('Week')['Sale'].sum().reset_index()
weekly_revenue_before['Week'] = weekly_revenue_before['Week'].dt.to_timestamp()

# Calculate weekly revenue after stage 1
org_df_after_stage_1['Week'] = org_df_after_stage_1['Date'].dt.to_period('W')
weekly_revenue_after = org_df_after_stage_1.groupby('Week')['Sale'].sum().reset_index()
weekly_revenue_after['Week'] = weekly_revenue_after['Week'].dt.to_timestamp()

# Create the first plot (before stage 1)
plt.figure(figsize=(12, 6))
plt.plot(weekly_revenue_before['Week'], weekly_revenue_before['Sale'], marker='o', linestyle='-', label='Weekly Revenue Before Stage 1')
plt.xlabel('Date')
plt.ylabel('Weekly Revenue')
plt.title(f'Weekly Revenue Before Stage 1 for {org_name}')
plt.grid(True)
plt.legend()
plt.show()

# Create the second plot (after stage 1)
plt.figure(figsize=(12, 6))
plt.plot(weekly_revenue_after['Week'], weekly_revenue_after['Sale'], marker='o', linestyle='-', label='Weekly Revenue After Stage 1')

# Mark stage changes on the second plot
stage_changes_org = ad[(ad['Organization'] == org_name) & (ad['Stage'].notna())].copy()
stage_change_dates = stage_changes_org['Date'].tolist()
stage_change_stages = stage_changes_org['Stage'].tolist()

legend_handles = []
for date, stage in zip(stage_change_dates, stage_change_stages):
    line = plt.axvline(date, color='r', linestyle='--', lw=1, label=f'Stage {int(stage)} Change')
    legend_handles.append(line)

plt.xlabel('Date')
plt.ylabel('Weekly Revenue')
plt.title(f'Weekly Revenue and Stage Changes After Stage 1 for {org_name}')
plt.grid(True)
plt.legend()
plt.show()


Display graphs from the top 10% of organizations who have progressed incrementally through changes. Creates a polynomial line of best fit and displays where the greatest increase in revenue was


In [ ]:
# Filter the df DataFrame to include only rows where the 'Stage' column is not NaN.
stage_changes = df[df['Stage'].notna()].copy()

# Group stage_changes by 'Organization' and count the number of stage changes.
stage_change_counts = stage_changes.groupby('Organization').size().reset_index(name='stage_change_count')

# Calculate the number of organizations corresponding to the top 10%.
num_unique_organizations = df['Organization'].nunique()
top_10_percent_count = int(num_unique_organizations * 0.1)

# Sort the grouped DataFrame by 'stage_change_count' in descending order.
sorted_stage_change_counts = stage_change_counts.sort_values(by='stage_change_count', ascending=False)

# Select the top top_10_percent_count organizations.
top_organizations_df = sorted_stage_change_counts.head(top_10_percent_count)

# Store their names in a list called top_organizations.
top_organizations = top_organizations_df['Organization'].tolist()

print(f"Number of unique organizations: {num_unique_organizations}")
print(f"Number of organizations in the top 10%: {top_10_percent_count}")
print("\nTop organizations by stage change count:")
print(top_organizations)

In [ ]:
from scipy.stats import linregress
import numpy as np
import matplotlib.pyplot as plt

for org_name in top_organizations:
    print(f"\n--- Analyzing {org_name} ---")

    org_df = df[df['Organization'] == org_name].copy()

    # Calculate weekly revenue for the entire period
    if not org_df.empty:
        org_df['Week'] = org_df['Date'].dt.to_period('W')
        weekly_revenue = org_df.groupby('Week')['Sale'].sum().reset_index()
        weekly_revenue['Week'] = weekly_revenue['Week'].dt.to_timestamp()

        # Convert datetime to numerical representation (timestamp)
        weekly_revenue['Week_numeric'] = weekly_revenue['Week'].apply(lambda x: x.timestamp())

        # Fit a polynomial regression model (degree 3 as a starting point)
        degree = 3
        coeffs = np.polyfit(weekly_revenue['Week_numeric'], weekly_revenue['Sale'], degree)
        poly_model = np.poly1d(coeffs)

        # Generate predicted revenue values
        weekly_revenue['Non_Linear_Fit'] = poly_model(weekly_revenue['Week_numeric'])

        # Calculate the derivative of the polynomial to find the slope
        poly_derivative = np.polyder(poly_model)

        # Calculate the slope at each point
        weekly_revenue['Slope'] = poly_derivative(weekly_revenue['Week_numeric'])

        # Find the point with the largest slope
        max_slope_row = weekly_revenue.loc[weekly_revenue['Slope'].idxmax()]

        # Create the plot
        plt.figure(figsize=(12, 6))
        plt.plot(weekly_revenue['Week'], weekly_revenue['Sale'], marker='o', linestyle='-', label='Weekly Revenue')
        plt.plot(weekly_revenue['Week'], weekly_revenue['Non_Linear_Fit'], color='purple', linestyle='--', linewidth=2, label='Non-Linear Best Fit')

        # Mark all stage changes on the plot
        stage_changes_org = ad[(ad['Organization'] == org_name) & (ad['Stage'].notna())].copy()

        # Get unique stage changes by date to avoid duplicate vertical lines at the same date
        unique_stage_changes = stage_changes_org.drop_duplicates(subset=['Date', 'Stage']).sort_values('Date')

        # Add a single legend entry for stage changes
        if not unique_stage_changes.empty:
            plt.axvline(unique_stage_changes.iloc[0]['Date'], color='r', linestyle='--', lw=1, label='Stage Change')
            for index, row in unique_stage_changes.iterrows():
                date = row['Date']
                # Only plot the vertical line without adding to the legend here
                plt.axvline(date, color='r', linestyle='--', lw=1)

        # Mark the point of largest slope on the best fit line
        plt.plot(max_slope_row['Week'], max_slope_row['Non_Linear_Fit'], marker='X', markersize=10, color='green', label=f'Max Slope Point (Slope: {max_slope_row["Slope"]:.2f})')


        plt.xlabel('Date')
        plt.ylabel('Weekly Revenue')
        plt.title(f'Weekly Revenue, Non-Linear Fit, and Stage Changes for {org_name}')
        plt.grid(True)
        plt.legend()
        plt.show()
    else:
        print(f"No data found for {org_name}")

Save as a CSV/Excel

In [ ]:
#if you need to download the df as excel

from google.colab import files

# Export to Excel
df.to_excel('cleaned_combined_data.xlsx', index=False)

# Download the file
files.download('cleaned_combined_data.xlsx')
print("✓ Download started!")